### Run in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1G4qV9OMj7E1XHc8F1qezJ6E0ird3K_7j#scrollTo=957LeMHjAdNY)

### Setup your API key

Before you can use the OpenAI API, you must first obtain an API key. If you don't already have one, create a key for OPENAI_API_KEY which is obtained from: https://openai.com/blog/openai-api

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `OPENAI_API_KEY`.


### Mount this Colab notebook at Google Drive
To ensure your work is saved and can be accessed later, it's a good practice to mount your Google Drive to this Colab notebook. You can do this by running the following code. This will prompt you to authorize Colab to access your Google Drive account. After authorization, your Google Drive will be mounted, and you can save your files directly to it.

In [ ]:
# Used to securely store your API key
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

# Mount google drive in the virtual envrionment
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ENVIRONMENT SETUP

In [ ]:
!pip install crewai
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Download Essential Data

If this is the first time you run the code, make sure you set download_data to True

In [ ]:
import os

# Specify the output folder where the CSV will be saved
output_folder = "/content/drive/MyDrive/LLM_crewai_RAG/"

# Check if the output folder exists and create it if it does not
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

# Google Drive download commands using gdown for each file
!gdown --id 1-UaVrPof8mhpZFtt958Ns7zRu-01-E-E -O "/content/drive/MyDrive/LLM_crewai_RAG/EngineeringReference.pdf"

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-UaVrPof8mhpZFtt958Ns7zRu-01-E-E
To: /content/drive/MyDrive/LLM_crewai_RAG/EngineeringReference_1.pdf
100% 23.8M/23.8M [00:00<00:00, 103MB/s] 


In [ ]:
from crewai import Agent, Task, Crew

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import Image

import openai
client = openai.OpenAI(api_key=api_key)

# Warning control
import warnings
warnings.filterwarnings('ignore')

from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

from crewai import Agent, Crew, Process, Task
from crewai_tools import PDFSearchTool
# from dotenv import load_dotenv
import os
# load_dotenv()

os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
# --- Tools ---
# PDF SOURCE: https://www.gpinspect.com/wp-content/uploads/2021/03/sample-home-report-inspection.pdf
pdf_search_tool = PDFSearchTool(
    pdf="/content/drive/MyDrive/LLM_crewai_RAG/EngineeringReference.pdf"
)

# --- Agents ---
research_agent = Agent(
    role="Research Agent",
    goal="Search through the PDF to find relevant answers",
    allow_delegation=False,
    verbose=True,
    backstory=(
        """
        The research agent is adept at searching and
        extracting data from documents, ensuring accurate and prompt responses.
        """
    ),
    tools=[pdf_search_tool],

)

professional_writer_agent = Agent(
    role="Professional Re-Writer",
    goal="Re-Write improved professional answer based on the research agent's findings, to make sure it is understandable by non-building expert",
    allow_delegation=False,
    verbose=True,
    backstory=(
        """
        The professional writer agent has excellent writing skills and is able to craft
        clear and concise response based on the provided information.
        """
    ),
    tools=[],
)


# --- Tasks ---
answer_people_question_task = Task(
    description=(
        """
        Answer the people's questions based on the content in PDF.
        The research agent will search through the PDF to find the relevant answers.
        Your final answer MUST be clear and accurate, based on the content of the PDF.

        Here is the people's question:
        {people_question}
        """
    ),
    expected_output="""
        Provide clear and accurate answers to the people's questions based on
        the content of the home inspection PDF.
        """,
    tools=[pdf_search_tool],
    agent=research_agent,
)

professional_writer_task = Task(
    description=(
        """
        Re-Write improved professional answer based on the research agent's findings, to make sure it is understandable by non-building expert by expalining terminologies

        """
    ),
    expected_output="""
        Provide improved writing clear and accurate answers based on the research agent's findings, to make sure it is understandable by non-building expert.
        """,
    tools=[],
    agent=professional_writer_agent,
)

# --- Crew ---
crew = Crew(
    agents=[research_agent, professional_writer_agent],
    tasks=[answer_people_question_task, professional_writer_task],
    process=Process.sequential,
)

people_question = "What is Baffle Heat Balance?\n"

result = crew.kickoff(inputs={"people_question": people_question})
print(result)

# Agent: Research Agent
## Task: 
        Answer the people's questions based on the content in PDF.
        The research agent will search through the PDF to find the relevant answers.
        Your final answer MUST be clear and accurate, based on the content of the PDF.

        Here is the people's question:
        What is Baffle Heat Balance?

        
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for FixedPDFSearchToolSchema
query
  Input should be a valid string [type=string_type, input_value={'description': 'Baffle H...Balance', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Search a PDF's content accepts these inputs: Tool Name: Search a PDF's content
Tool Arguments: {'query': {'description': "Mandatory query you want to use to search the PDF's content", 'type': 'str'}}
Tool Description: A tool that can be used to semantic sear